<a href="https://colab.research.google.com/github/nithyaanantharamanGIT/DATA226_Datawarehouse/blob/main/DATA226_HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Java and Spark dependencies
!apt-get update
!apt-get install -y openjdk-11-jdk-headless

# Install PySpark and findspark
!pip install pyspark findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,151 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,475 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,532 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy

In [3]:
# Set Java home and import findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [4]:
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_1.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_2.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_3.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_4.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_5.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_6.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_7.log.gz

--2025-11-22 07:03:14--  https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_1.log.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10277393 (9.8M) [application/octet-stream]
Saving to: ‘sample_web_log_1.log.gz’

sample_web_log_1.lo 100%[===================>]   9.80M  --.-KB/s    in 0.1s    

2025-11-22 07:03:15 (100 MB/s) - ‘sample_web_log_1.log.gz’ saved [10277393/10277393]

--2025-11-22 07:03:15--  https://raw.githubusercontent.com/keeyong/sjsu-data226-FA25/refs/heads/main/week13/data/sample_web_log_2.log.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.

## Configure snowflake jar file and set up SparkSession

In [5]:
!cd /usr/local/lib/python3.12/dist-packages/pyspark/jars && wget https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar

--2025-11-22 07:03:21--  https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.18.12, 104.18.19.12, 2606:4700::6812:130c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.18.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70986770 (68M) [application/java-archive]
Saving to: ‘snowflake-jdbc-3.19.0.jar’

snowflake-jdbc-3.19 100%[===================>]  67.70M  92.8MB/s    in 0.7s    

2025-11-22 07:03:22 (92.8 MB/s) - ‘snowflake-jdbc-3.19.0.jar’ saved [70986770/70986770]



In [6]:
# Now you can use PySpark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("HandleLogFiles").getOrCreate()

## Create an input dataframe

In [7]:
# Load all .gz files in the directory into a DataFrame
df = spark.read.text("*.gz")

In [8]:
# Check the number of partitions
print(df.rdd.getNumPartitions())

df.show(truncate=False)

3
+-----------------------------------------------------------------------------------+
|value                                                                              |
+-----------------------------------------------------------------------------------+
|123.45.67.89 - - [05/Nov/2024:02:08:16 +0000] "DELETE /cart HTTP/1.1" 500 242      |
|192.168.1.1 - - [04/Nov/2024:21:23:39 +0000] "POST /checkout HTTP/1.1" 404 2781    |
|234.56.78.90 - - [05/Nov/2024:07:06:19 +0000] "GET /api/data HTTP/1.1" 301 3758    |
|192.168.1.1 - - [04/Nov/2024:20:03:56 +0000] "POST /home HTTP/1.1" 200 1837        |
|192.168.1.1 - - [04/Nov/2024:21:25:05 +0000] "GET /products/123 HTTP/1.1" 200 3430 |
|234.56.78.90 - - [04/Nov/2024:07:38:10 +0000] "GET /api/data HTTP/1.1" 404 3729    |
|123.45.67.89 - - [04/Nov/2024:12:33:22 +0000] "PUT /api/data HTTP/1.1" 404 799     |
|192.168.1.1 - - [04/Nov/2024:07:37:46 +0000] "GET /api/data HTTP/1.1" 500 309      |
|123.45.67.89 - - [04/Nov/2024:21:52:36 +0000] "POST

## Create a parsed dataframe (log_df)

In [9]:
# Extract the necessary information from log data using regular expressions
pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?) (.*?) HTTP.*" (\d+) (\d+)'

log_df = df.select(
    F.regexp_extract("value", pattern, 1).alias("ip"),
    F.regexp_extract("value", pattern, 2).alias("timestamp"),
    F.regexp_extract("value", pattern, 3).alias("method"),
    F.regexp_extract("value", pattern, 4).alias("url"),
    F.regexp_extract("value", pattern, 5).alias("status").cast("integer"),
    F.regexp_extract("value", pattern, 6).alias("size").cast("integer")
)

In [10]:
log_df.show()

+------------+--------------------+------+-------------+------+----+
|          ip|           timestamp|method|          url|status|size|
+------------+--------------------+------+-------------+------+----+
|123.45.67.89|05/Nov/2024:02:08...|DELETE|        /cart|   500| 242|
| 192.168.1.1|04/Nov/2024:21:23...|  POST|    /checkout|   404|2781|
|234.56.78.90|05/Nov/2024:07:06...|   GET|    /api/data|   301|3758|
| 192.168.1.1|04/Nov/2024:20:03...|  POST|        /home|   200|1837|
| 192.168.1.1|04/Nov/2024:21:25...|   GET|/products/123|   200|3430|
|234.56.78.90|04/Nov/2024:07:38...|   GET|    /api/data|   404|3729|
|123.45.67.89|04/Nov/2024:12:33...|   PUT|    /api/data|   404| 799|
| 192.168.1.1|04/Nov/2024:07:37...|   GET|    /api/data|   500| 309|
|123.45.67.89|04/Nov/2024:21:52...|  POST|    /checkout|   301|2375|
|123.45.67.89|04/Nov/2024:08:36...|DELETE|    /api/data|   404|3449|
| 192.168.1.1|05/Nov/2024:03:15...|   GET|    /api/data|   200|2319|
|234.56.78.90|05/Nov/2024:01:26...

In [21]:
# Extract the necessary information from log data using regular expressions
pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?) (.*?) HTTP.*" (\d+) (\d+)'

log_df = df.select(
    F.regexp_extract("value", pattern, 1).alias("ip"),
    F.regexp_extract("value", pattern, 3).alias("method")
)
log_df.show()

+------------+------+
|          ip|method|
+------------+------+
|123.45.67.89|DELETE|
| 192.168.1.1|  POST|
|234.56.78.90|   GET|
| 192.168.1.1|  POST|
| 192.168.1.1|   GET|
|234.56.78.90|   GET|
|123.45.67.89|   PUT|
| 192.168.1.1|   GET|
|123.45.67.89|  POST|
|123.45.67.89|DELETE|
| 192.168.1.1|   GET|
|234.56.78.90|DELETE|
|234.56.78.90|DELETE|
|123.45.67.89|   PUT|
|123.45.67.89|   GET|
|123.45.67.89|DELETE|
|123.45.67.89|  POST|
|234.56.78.90|   GET|
| 192.168.1.1|  POST|
|234.56.78.90|   PUT|
+------------+------+
only showing top 20 rows



In [22]:
ip_method_count = log_df.groupBy("ip", "method").count().orderBy("count", ascending=False)
ip_method_count.show()


+------------+------+------+
|          ip|method| count|
+------------+------+------+
|234.56.78.90|   GET|584382|
|123.45.67.89|  POST|584073|
| 192.168.1.1|   GET|583997|
|234.56.78.90|  POST|583852|
|123.45.67.89|   PUT|583654|
|123.45.67.89|   GET|583590|
| 192.168.1.1|DELETE|583384|
|234.56.78.90|   PUT|583383|
|234.56.78.90|DELETE|582585|
| 192.168.1.1|  POST|582488|
| 192.168.1.1|   PUT|582412|
|123.45.67.89|DELETE|582200|
+------------+------+------+



## Now Let's do this in SparkSQL

In [23]:
log_df.createOrReplaceTempView("logs")
ip_method_count_sql = spark.sql("""
    SELECT ip, method, COUNT(*) as count
    FROM logs
    GROUP BY ip, method
    ORDER BY count DESC
""")
ip_method_count_sql.show()


+------------+------+------+
|          ip|method| count|
+------------+------+------+
|234.56.78.90|   GET|584382|
|123.45.67.89|  POST|584073|
| 192.168.1.1|   GET|583997|
|234.56.78.90|  POST|583852|
|123.45.67.89|   PUT|583654|
|123.45.67.89|   GET|583590|
| 192.168.1.1|DELETE|583384|
|234.56.78.90|   PUT|583383|
|234.56.78.90|DELETE|582585|
| 192.168.1.1|  POST|582488|
| 192.168.1.1|   PUT|582412|
|123.45.67.89|DELETE|582200|
+------------+------+------+



## Let's save this DF as a table in Snowflake

In [26]:
from google.colab import userdata

account = userdata.get('snowflake_account')
user = userdata.get('snowflake_userid')
password = userdata.get('snowflake_password')
database = userdata.get('snowflake_database')
schema = "analytics"

url = f"jdbc:snowflake://{account}.snowflakecomputing.com/?db={database}&schema={schema}&user={user}&password={password}"

In [27]:
ip_method_count.write \
    .format("jdbc") \
    .option("driver", "net.snowflake.client.jdbc.SnowflakeDriver") \
    .option("url", url) \
    .mode("overwrite") \
    .option("dbtable", "ip_method_count") \
    .save()